Repeat the individual transect level MRDI analysis, but with indigenous and introduced species separate.

In [1]:
import numpy as np
import pandas as pd
import scipy.stats as st
import matplotlib.pyplot as plt
import sad_mrdi as sm
%matplotlib inline

In [2]:
# Import data
ad = pd.read_csv('./RawData/Azores_Adults.csv',header=[0,1])

# Now separate out
# Strip extra whitespace
ad['Data','N/E/I'] = ad['Data','N/E/I'].str.strip()
# Get indices for N/E
indigenous_inds = np.any([ad['Data','N/E/I'] =='N',ad['Data','N/E/I'] == 'E'],axis=0)
introduced_inds = (ad['Data','N/E/I'] =='I').values

# Check how many aren't categorized
print("Not categorized: {}".format(len(ad)-len(ad[indigenous_inds])-len(ad[introduced_inds])))

Not categorized: 2


In [3]:
# More preamble
# Some more preamble and calculating some state variables
# Get total s0
s0 = len(ad)
print('Number of species: {}'.format(s0)) 
lu = list(ad.columns.levels[0])
lu.remove('Data')

# Get length to use to loop over etc.
lutypes = len(lu)
# Get how many sites for each land use
lu_sites = pd.Series(index=lu,dtype=int)
for l in lu:
    lu_sites[l] = len(ad[l].columns)

# Reorder to disturbance gradient
lu = [lu[2],lu[0],lu[3],lu[1]]
# Get total n0
n0 = ad[lu].sum().sum()
print('Number of individuals: {}'.format(n0))

# How many indigenous versus introduced species?
n0_indigenous = np.sum(indigenous_inds)
n0_introduced = np.sum(introduced_inds)
print('Number of indigenous species: {}'.format(n0_indigenous))
print('Number of introduced species: {}'.format(n0_introduced))
# Note now only 2 species aren't defined, since we have fewer species in this dataset.

Number of species: 226
Number of individuals: 36269
Number of indigenous species: 98
Number of introduced species: 126


In [4]:
# Get list of sites, ignoring first 9 indices which are data information
ls = ad.columns[9:]

# Make arrays using ls instead of lu
# For s,n,beta. Later e0 and lambdas.
sne_idg = pd.DataFrame(index=ls,columns = {'s0','n0','beta'})
sne_int = pd.DataFrame(index=ls,columns = {'s0','n0','beta'})
# For abundances
abd_idg = pd.DataFrame(columns=ls)
abd_int = pd.DataFrame(columns=ls)
for l in ls:
    abd_idg[l] = ad[l].iloc[indigenous_inds]
    abd_int[l] = ad[l].iloc[introduced_inds]
    
    # Indigenous
    # Get n0 and s0
    stemp = np.count_nonzero(abd_idg[l])
    ntemp = abd_idg[l].sum()
    # Get beta
    btemp = sm.get_beta(stemp,ntemp)
    # Add to dataframe
    sne_idg.loc[l] = {'n0': ntemp, 's0': stemp, 'beta': btemp}
    
    # Introduced
    # Get n0 and s0
    stemp = np.count_nonzero(abd_int[l])
    ntemp = abd_int[l].sum()
    # Get beta
    btemp = sm.get_beta(stemp,ntemp)
    # Add to dataframe
    sne_int.loc[l] = {'n0': ntemp, 's0': stemp, 'beta': btemp}

# Rename indexes for abundaces to species code
abd_idg.rename(index=ad['Data','MF'],inplace=True)
abd_idg.index.name = 'MF'
abd_int.rename(index=ad['Data','MF'],inplace=True)
abd_int.index.name = 'MF'

# Fix datatype for sn_lu
sne_idg = sne_idg.astype({'s0': 'int64','n0':'int64','beta':'float64'})
sne_int = sne_int.astype({'s0': 'int64','n0':'int64','beta':'float64'})

/Users/micbru/Documents/Research/2020Azores/CleanCode/sad_mrdi.py:41: RuntimeWarning: invalid value encountered in double_scalars
  return expnb.sum()/(expnb/nrange).sum()-N/S
/Users/micbru/Documents/Research/2020Azores/CleanCode/sad_mrdi.py:41: RuntimeWarning: invalid value encountered in long_scalars
  return expnb.sum()/(expnb/nrange).sum()-N/S
/Users/micbru/anaconda3/lib/python3.6/site-packages/scipy/optimize/minpack.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


# Simulate datasets

In [5]:
# Set up regression relationships from BodyMassVariance.ipynb
# Note these are for the log relationship
# For beetles
bi = -1.243073857459273
bs = 1.9948767678521848
# For spiders
si = -1.1467463900692998
ss = 2.2207391333864335

In [6]:
# Set random seed
prng = np.random.RandomState(101)
# Make a dictionary with labeled land uses
biomass_idg = {}
biomass_int = {}
for l in ls:
    # Indigenous
    # Find args for this land use where the abundance is non-zero
    args_temp = np.where(abd_idg[l])[0]
    # Get abundances just for this site
    abd_temp = abd_idg[l].iloc[args_temp]
    # Now simulate that number of points for each species
    biomass_idg[l] = np.array([])
    for mf in abd_temp.index:
        # pull abd_temp[mf] number of points from a normal distribution
        # where the mean is given by the mean in the main dataset
        # and the standard deviation is given by the regression relationships above
        mean = ad[ad['Data','MF']==mf]['Data','Body_Mass.mg.']
        # Use the beetle regression
        if (ad[ad['Data','MF']==mf]['Data','Order (new)'].values[0]=='Araneae'):
            var = 10**(si+ss*np.log10(mean))
        else:
            var = 10**(bi+bs*np.log10(mean))
        biomass_idg[l] = np.append(biomass_idg[l],
                               st.norm.rvs(loc=mean,scale=np.sqrt(var),size=abd_temp[mf],random_state=prng))
    # Introduced
    # Find args for this land use where the abundance is non-zero
    args_temp = np.where(abd_int[l])[0]
    # Get abundances just for this site
    abd_temp = abd_int[l].iloc[args_temp]
    # Now simulate that number of points for each species
    biomass_int[l] = np.array([])
    for mf in abd_temp.index:
        # pull abd_temp[mf] number of points from a normal distribution
        # where the mean is given by the mean in the main dataset
        # and the standard deviation is given by the regression relationships above
        mean = ad[ad['Data','MF']==mf]['Data','Body_Mass.mg.']
        # Use the beetle regression
        if (ad[ad['Data','MF']==mf]['Data','Order (new)'].values[0]=='Araneae'):
            var = 10**(si+ss*np.log10(mean))
        else:
            var = 10**(bi+bs*np.log10(mean))
        biomass_int[l] = np.append(biomass_int[l],
                               st.norm.rvs(loc=mean,scale=np.sqrt(var),size=abd_temp[mf],random_state=prng))

In [7]:
# Fix the smallest ones and remove problem points
for l in ls:
    inds = biomass_idg[l]<0
    if np.any(inds):
        print("Problem point",biomass_idg[l][biomass_idg[l]<0])
        # To avoid duplicates, add a tiny noise to this.
        biomass_idg[l][inds] = np.min(biomass_idg[l][~inds])*(1+0.01*st.norm.rvs(random_state=prng))
        print("Should be empty",biomass_idg[l][biomass_idg[l]<0])
        print("Previous minimum",np.min(biomass_idg[l][~inds]))
        print("Newly added",biomass_idg[l][inds])
        print()
    inds = biomass_int[l]<0
    if np.any(inds):
        print("Problem point",biomass_int[l][biomass_int[l]<0])
        # To avoid duplicates, add a tiny noise to this.
        biomass_int[l][inds] = np.min(biomass_int[l][~inds])*(1+0.01*st.norm.rvs(random_state=prng))
        print("Should be empty",biomass_int[l][biomass_int[l]<0])
        print("Previous minimum",np.min(biomass_int[l][~inds]))
        print("Newly added",biomass_int[l][inds])
        print()

Problem point [-0.13374193]
Should be empty []
Previous minimum 0.09132743984038377
Newly added [0.09327471]



In [8]:
# Now convert to metabolic rate
mr_idg = biomass_idg.copy()
mr_int = biomass_int.copy()
for l in ls:
    # Indigenous
    if mr_idg[l].size!=0:
        # Now sort the array, convert to metabolic rate (m \propto e^(4/3)), and divide by smallest
        # Order doesn't matter here
        mr_idg[l] = np.sort(mr_idg[l]**(3/4))
        # Note that this way, the e0 between the land uses actually isn't comparable 
        #because the smallest unit is different
        mr_idg[l] = mr_idg[l]/mr_idg[l][0]
    
    # Introduced
    # Need a catch because there is a size 0 in one site
    if mr_int[l].size!=0:
        mr_int[l] = np.sort(mr_int[l]**(3/4))
        mr_int[l] = mr_int[l]/mr_int[l][0]
    
# Get E0
for l in ls:
    sne_idg.loc[l,'e0'] = mr_idg[l].sum()
    sne_int.loc[l,'e0'] = mr_int[l].sum()

sne_idg['l2'] = sne_idg['s0']/(sne_idg['e0']-sne_idg['n0'])
sne_idg['l1'] = sne_idg['beta']-sne_idg['l2']

sne_int['l2'] = sne_int['s0']/(sne_int['e0']-sne_int['n0'])
sne_int['l1'] = sne_int['beta']-sne_int['l2']

In [9]:
# Check for duplicates
for l in ls:
    if len(np.unique(mr_idg[l])) != len(mr_idg[l]):
        print("Ruh roh!")
    if len(np.unique(mr_int[l])) != len(mr_int[l]):
        print("Ruh roh!")

# LEAST SQUARES
Let's just do this for the indigenous/introduced so as not to have a billion plots.

## Goodness of fit and summary plots

In [10]:
# Use mean least squares
mlsq_idg = pd.Series(index=ls,dtype='float64')
mlsq_int = pd.Series(index=ls,dtype='float64')
for l in ls:
    # Indigenous
    ranks = np.arange(sne_idg.loc[l,'n0'])+1
    pred = sm.mrdi_rank(ranks,(sne_idg.loc[l,'l1'],sne_idg.loc[l,'l2']),sne_idg.loc[l,'n0'])
    obs = mr_idg[l][::-1]
    # If we have any less than 0, ignore that because n0 is too small.
    if np.any(pred <= 0):
        mlsq_idg[l] = np.nan
    elif sne_idg['n0'][l] == len(obs):
        mlsq_idg[l] = np.sum((np.log(obs)-np.log(pred))**2)/len(obs)
    else:
        print("Ruh ro!")
        
    # Introduced
    ranks = np.arange(sne_int.loc[l,'n0'])+1
    pred = sm.mrdi_rank(ranks,(sne_int.loc[l,'l1'],sne_int.loc[l,'l2']),sne_int.loc[l,'n0'])
    obs = mr_int[l][::-1]
    # If we have any less than 0, ignore that because n0 is too small.
    if np.any(pred <= 0):
        mlsq_int[l] = np.nan
    elif sne_int['n0'][l] == len(obs):
        mlsq_int[l] = np.sum((np.log(obs)-np.log(pred))**2)/len(obs)
    else:
        print("Ruh ro!")

# modify number of sites with number of nans
lu_sites_alt = lu_sites.copy()
lu_sites_alt_idg = lu_sites_alt - mlsq_idg.isna().sum(level=0)
lu_sites_alt_int = lu_sites_alt - mlsq_int.isna().sum(level=0)
        
sne_idg['mlsq'] = mlsq_idg
sne_int['mlsq'] = mlsq_int

/Users/micbru/Documents/Research/2020Azores/CleanCode/sad_mrdi.py:79: RuntimeWarning: invalid value encountered in double_scalars
  logarg = ((np.exp(l[0]+l[1])-1)*N+r-1/2)/(r-1/2)
/Users/micbru/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in double_scalars
/Users/micbru/Documents/Research/2020Azores/CleanCode/sad_mrdi.py:80: RuntimeWarning: invalid value encountered in log
  return (np.log(logarg) - l[0])/l[1]


In [11]:
# Note that .std in pandas already has ddof=1, which is correct here since we estimate the mean
# To get the standard error of the mean, we have to divide by sqrt(n)
mean_idg = sne_idg['mlsq'].mean(level=0)
mean_int = sne_int['mlsq'].mean(level=0)
display(mean_idg)
display(mean_int)

se_idg = sne_idg['mlsq'].std(level=0)
se_int = sne_int['mlsq'].std(level=0)
for l in lu:
    se_idg.loc[l] /= np.sqrt(lu_sites_alt_idg[l])
    se_int.loc[l] /= np.sqrt(lu_sites_alt_int[l])
display(se_idg)
display(se_int)

Exotic forest           1.320969
Native forest           0.741922
Intensive pasture       1.398870
Semi-natural pasture    0.790244
Name: mlsq, dtype: float64

Exotic forest           1.454388
Native forest           2.021504
Intensive pasture       0.904025
Semi-natural pasture    0.790737
Name: mlsq, dtype: float64

Exotic forest           0.191097
Native forest           0.086812
Intensive pasture       0.381347
Semi-natural pasture    0.203942
Name: mlsq, dtype: float64

Exotic forest           0.209786
Native forest           0.291581
Intensive pasture       0.100410
Semi-natural pasture    0.158950
Name: mlsq, dtype: float64

In [12]:
# save the means to file
mlsq_data = pd.DataFrame([mean_idg,se_idg,lu_sites_alt_idg,mean_int,se_int,lu_sites_alt_int],
                         index=['Mean (idg)','Standard error (idg)','N (idg)',
                                'Mean (int)','Standard error (int)','N (int)'])
display(mlsq_data)
mlsq_data.to_csv('ProcessedData/mrdi_mlsq_indigenous.csv')

# Also save raw data
#sne_idg.to_csv('ProcessedData/mrdi_indigenous.csv')
#sne_int.to_csv('ProcessedData/mrdi_introduced.csv')

,Exotic forest,Native forest,Intensive pasture,Semi-natural pasture
Mean (idg),1.320969,0.741922,1.398870,0.790244
Standard error (idg),0.191097,0.086812,0.381347,0.203942
N (idg),11.000000,44.000000,24.000000,15.000000
Mean (int),1.454388,2.021504,0.904025,0.790737
Standard error (int),0.209786,0.291581,0.100410,0.158950
N (int),12.000000,32.000000,24.000000,16.000000
